# 🧮 Odvození stavového modelu inverzního kyvadla na vozíku

## 1️⃣ Parametry a proměnné

- $m_1$ ... hmotnost vozíku  
- $m_2$ ... hmotnost kyvadla  
- $l$ ... délka tyče  
- $g$ ... gravitační konstanta  
- $x$ ... poloha vozíku  
- $\theta$ ... úhel kyvadla od svislice směrem nahoru (rovnováha je v bodě $\theta = \pi$)  
- $F$ ... síla působící na vozík  

Stavový vektor:

$$
\mathbf{x} =
\begin{bmatrix}
x \\
\dot{x} \\
\theta \\
\dot{\theta}
\end{bmatrix}
$$

Řídicí vstup:

$$
u = F
$$

---

## 2️⃣ Nelineární rovnice (Newtonovy zákony)

Získáme soustavu nelineárních rovnic:

$$
\begin{aligned}
(m_1 + m_2) \ddot{x} - m_2 l \ddot{\theta} \cos(\theta) + m_2 l \dot{\theta}^2 \sin(\theta) &= F \\
l \ddot{\theta} - \ddot{x} \cos(\theta) - g \sin(\theta) &= 0
\end{aligned}
$$

---

## 3️⃣ Linearizace kolem rovnováhy $\theta = \pi$

Použijeme Taylorův rozvoj kolem bodu $\theta = \pi$:

$$
\begin{aligned}
\sin(\theta) &\approx -(\theta - \pi) \\
\cos(\theta) &\approx -1 \\
\dot{\theta}^2 &\approx 0
\end{aligned}
$$

Zavedeme novou proměnnou:

$$
\tilde{\theta} = \theta - \pi
$$

Dosadíme do rovnic a zanedbáme vyšší členy:

### První rovnice:

$$
(m_1 + m_2) \ddot{x} + m_2 l \ddot{\tilde{\theta}} \approx F
\Rightarrow \ddot{x} = \frac{1}{m_1 + m_2} \left(F - m_2 l \ddot{\tilde{\theta}} \right)
$$

### Druhá rovnice:

$$
l \ddot{\tilde{\theta}} + \ddot{x} + g \tilde{\theta} = 0
\Rightarrow \ddot{\tilde{\theta}} = -\frac{\ddot{x}}{l} - \frac{g}{l} \tilde{\theta}
$$

Dosadíme $\ddot{x}$ z předchozí rovnice:

$$
\ddot{\tilde{\theta}} = -\frac{1}{l} \cdot \left( \frac{1}{m_1 + m_2} (F - m_2 l \ddot{\tilde{\theta}}) \right) - \frac{g}{l} \tilde{\theta}
$$

Vynásobíme a upravíme:

$$
\ddot{\tilde{\theta}} + \frac{m_2}{m_1 + m_2} \ddot{\tilde{\theta}} = -\frac{F}{l(m_1 + m_2)} - \frac{g}{l} \tilde{\theta}
$$

Sečteme členy:

$$
\ddot{\tilde{\theta}} \left(1 + \frac{m_2}{m_1 + m_2}\right) = - \frac{F}{l(m_1 + m_2)} - \frac{g}{l} \tilde{\theta}
$$

---

## 4️⃣ Stavový model

Výsledný lineární model ve tvaru $\dot{\mathbf{x}} = A \mathbf{x} + B u$ je:

$$
A =
\begin{bmatrix}
0 & 1 & 0 & 0 \\
0 & 0 & -\frac{m_2 g}{m_1} & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & \frac{(m_1 + m_2) g}{l m_1} & 0
\end{bmatrix},
\quad
B =
\begin{bmatrix}
0 \\
\frac{1}{m_1} \\
0 \\
-\frac{1}{l m_1}
\end{bmatrix}
$$

---

## 5️⃣ Závěr

Tento stavový model je vhodný pro řízení metodou LQR k dosažení stabilizace kyvadla v horní poloze. Matice $A$ a $B$ lze dosadit do simulace nebo návrhu regulátoru.


# 🧠 Odvození LQR řízení pro inverzní kyvadlo

## 1️⃣ Cíl řízení

Chceme navrhnout řídicí zákon:

$$
u = -K \mathbf{x}
$$

tak, aby systém:

$$
\dot{\mathbf{x}} = A\mathbf{x} + B u
$$

byl stabilní a zároveň minimalizoval **kvadratickou cílovou funkci (cost function)**:

$$
J = \int_0^\infty \left( \mathbf{x}^\top Q \mathbf{x} + u^\top R u \right) dt
$$

- $Q \succeq 0$: váhová matice pro stavové odchylky  
- $R \succ 0$: váhová matice pro vstup (sílu)

---

## 2️⃣ Vzorec pro ziskovou matici K

LQR řízení minimalizuje $J$ při použití zpětné vazby $u = -K \mathbf{x}$, kde:

$$
K = R^{-1} B^\top P
$$

a matice $P$ je řešením **Algebraické Riccatiho rovnice**:

$$
A^\top P + P A - P B R^{-1} B^\top P + Q = 0
$$

---

## 3️⃣ Význam vah Q a R

- Matice **Q** určuje, které stavy chceme penalizovat více (např. úhel kyvadla).  
  Často se volí diagonální:

$$
Q = \mathrm{diag}(q_1, q_2, q_3, q_4)
$$

- Matice **R** penalizuje velikost řídicí síly $u$ – čím větší $R$, tím menší síla bude aplikována.

---

## 4️⃣ Praktický výběr Q a R

Příklad:

```python
Q = np.diag([10, 1, 100, 1])  # penalizuj odchylku polohy a hlavně úhlu
R = np.array([[1]])          # penalizuj sílu


In [2]:
# === 🔨 Regulace inverzního kyvadla pomocí LQR (stabilizace kolem theta = pi) ===
import numpy as np
from scipy.integrate import odeint
from scipy.linalg import solve_continuous_are
from sympy import symbols, simplify, cos, sin, solve, lambdify
import pygame

# === Parametry systému ===
m1V = 10  # hmotnost vozíku
m2V = 1  # hmotnost kyvadla
lV = 1  # délka tyče
gV = 9.81  # gravitace

# === Symbolické rovnice systému ===
m1, m2, l, g = symbols("m1 m2 l g")
F = symbols("F")
z1, z2, z3, z4 = symbols("z1 z2 z3 z4")
dz2, dz4 = symbols("dz2 dz4")

# Rovnice dynamiky
e1 = (m1 + m2) * dz2 - m2 * l * dz4 * cos(z3) + m2 * l * z4**2 * sin(z3) - F
e2 = l * dz4 - dz2 * cos(z3) - g * sin(z3)

# Vyřešíme rovnice
solved = solve([e1, e2], [dz2, dz4], dict=True)[0]
dz2_expr = simplify(solved[dz2])
dz4_expr = simplify(solved[dz4])

# Dosadíme číselné hodnoty
subs = {m1: m1V, m2: m2V, l: lV, g: gV}
dz2_expr = dz2_expr.subs(subs)
dz4_expr = dz4_expr.subs(subs)

# Funkce pro odeint
functionDz2 = lambdify([z1, z2, z3, z4, F], dz2_expr, modules="numpy")
functionDz4 = lambdify([z1, z2, z3, z4, F], dz4_expr, modules="numpy")

# === LINEARIZACE kolem theta = pi (inverzní poloha) ===
A = np.array([[0, 1, 0, 0], [0, 0, (m2V * gV) / m1V, 0], [0, 0, 0, 1], [0, 0, -((m1V + m2V) * gV) / (lV * m1V), 0]])

B = np.array([[0], [1 / m1V], [0], [-1 / (lV * m1V)]])

# === LQR DESIGN ===
Q = np.diag([10, 1, 300, 10])
R = np.array([[1]])

P = solve_continuous_are(A, B, Q, R)
K = np.linalg.inv(R) @ B.T @ P
K = K.flatten()


# === Model pro simulaci ===
def stateSpaceModel(z, t):
    x, x_dot, theta, theta_dot = z
    # přepočet na odchylku od theta = pi
    theta_error = theta - np.pi
    state_vec = np.array([x, x_dot, theta_error, theta_dot])
    u = -K @ state_vec
    dz2_val = functionDz2(x, x_dot, theta, theta_dot, u)
    dz4_val = functionDz4(x, x_dot, theta, theta_dot, u)
    return [x_dot, dz2_val, theta_dot, dz4_val]


# === Spuštění simulace ===
initial_state = np.array([0.0, 0.0, np.pi + np.deg2rad(5), 0.0])  # 5° odchylka od horní polohy
t = np.linspace(0, 10, 1000)
solution = odeint(stateSpaceModel, initial_state, t)

# === 🎥 ANIMACE pomocí pygame ===
pygame.init()
size = width, height = 1600, 800
screen = pygame.display.set_mode(size)
clock = pygame.time.Clock()

# data
x_data = solution[:, 0]
theta_data = solution[:, 2]

# škálování pozice vozíku na obrazovku
maxX, minX = max(x_data), min(x_data)
offsetScreenLimits = 500
lB = offsetScreenLimits
uB = width - offsetScreenLimits
scaleX = (uB - lB) / (maxX - minX)
offsetX = lB - scaleX * minX
x_pixels = scaleX * x_data + offsetX

# geometrie
ballRadius = 40
cartWidth = 150
cartHeight = 100
rodLength = 300
wheelRadius = 25
pendulumSupportRadius = 15
yPositionCart = 400

# barvy
colorRail = (255, 165, 0)
colorCart = (255, 255, 153)
colorBall = (255, 0, 0)
colorWheels = (0, 200, 0)
colorPendulumSupport = (255, 165, 0)
colorRod = (255, 0, 255)

# animace
i = 0
while i < len(x_pixels):
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            i = len(x_pixels)

    screen.fill((0, 0, 0))

    xC = x_pixels[i]
    yC = yPositionCart

    pygame.draw.line(
        screen,
        colorRail,
        (int(min(x_pixels) - 300), int(yC + cartHeight + 2 * wheelRadius)),
        (int(max(x_pixels) + 300), int(yC + cartHeight + 2 * wheelRadius)),
        6,
    )

    pygame.draw.rect(screen, colorCart, (int(xC - cartWidth / 2), yC, cartWidth, cartHeight))
    pygame.draw.circle(screen, colorPendulumSupport, (int(xC), int(yC)), pendulumSupportRadius)

    pygame.draw.circle(
        screen, colorWheels, (int(xC - cartWidth / 2 + wheelRadius), int(yC + cartHeight + wheelRadius)), wheelRadius
    )
    pygame.draw.circle(
        screen, colorWheels, (int(xC + cartWidth / 2 - wheelRadius), int(yC + cartHeight + wheelRadius)), wheelRadius
    )

    theta = theta_data[i]
    xB = xC + rodLength * np.sin(theta - np.pi)
    yB = yC - rodLength * np.cos(theta - np.pi)

    pygame.draw.line(screen, colorRod, (int(xC), yC), (int(xB), int(yB)), 8)
    pygame.draw.circle(screen, colorBall, (int(xB), int(yB)), ballRadius)

    pygame.display.flip()
    pygame.time.delay(1)
    clock.tick(100)
    i += 1

pygame.quit()
